In [ ]:
#If you have chosen to install webdriver_manager, then execute the following lines of code

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window()
driver.get("https://www.google.com/")
driver.close()


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

# List of gene names to search
gene_names = ["MDM4","EZH2","CDC42","MAX"]

# Set up Chrome options to ignore certificate errors
chrome_options = Options()
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--ignore-ssl-errors')

# Set up the Chrome driver
service = ChromeService(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.maximize_window()

# Open the website
driver.get("https://midb.pnb.uconn.edu/basic_search_php.php?species_var=Homo+sapiens&gene=CDK4&gene_class=all")

# Handle the "Your connection is not private" warning
try:
    # Click the "Advanced" button
    advanced_button = driver.find_element(By.XPATH, "//*[@id='details-button']")
    advanced_button.click()
    time.sleep(1)

    # Click the "Proceed to site" link
    proceed_link = driver.find_element(By.XPATH, "//*[@id='proceed-link']")
    proceed_link.click()
    time.sleep(2)
except Exception as e:
    print(f"No SSL warning bypass needed: {e}")

# Open a file to write the output
with open("gene_classifications.txt", "w") as file:
    for gene in gene_names:
        try:
            # Find the search bar and enter the gene name
            search_bar = driver.find_element(By.XPATH, "//input[@name='gene']")
            search_bar.clear()
            search_bar.send_keys(gene)
            
            # Click the search button
            search_button = driver.find_element(By.XPATH, "//button[@type='submit']")
            search_button.click()
            
            # Wait for the results to load
            time.sleep(2)
            
            # Find all rows in the table body
            table_body = driver.find_element(By.XPATH, '//*[@id="search"]/tbody')
            rows = table_body.find_elements(By.XPATH, './/tr')
            found = False
            
            for row in rows:
                cells = row.find_elements(By.XPATH, './/td')
                if cells:
                    gene_elements = row.find_elements(By.XPATH, './td[3]/a')
                    if gene_elements:
                        gene_name = gene_elements[0].text
                        if gene_name == gene:
                            gene_elements[0].click()
                            time.sleep(2)
                            
                            # Get the output string from the new page
                            output_element = driver.find_element(By.XPATH, '/html/body/div[6]/div[2]/div[3]')
                            output_string = output_element.text
                            found = True
                            break
            
            if not found:
                output_string = "not found"
            
            # Write the gene and output to the file
            file.write(f"{gene} {output_string}\n")
            print(f"Gene: {gene} - Output: {output_string}")
            
            # Navigate back to the search page for the next gene
            driver.back()
            time.sleep(2)  # Wait for the page to load before next iteration

        except Exception as e:
            print(f"An error occurred while processing gene {gene}: {e}")

# Close the driver
driver.quit()
